# Remote testing with Fabric
30/12/19

Currently have commands routed through Fabric for both local & remote running. On remote machines shell scripts invoking Anaconda work OK, but not on local. Issue with Fabric/Invoke, or machine settings...?  Debug here, and include pertinent notes.

In [1]:
import sys, os
from pathlib import Path

# Load dev scripts
modPath = r'/home/femtolab/python/'
sys.path.append(modPath)
import epsman as em

In [2]:
job = em.epsJob(host = 'localhost', IP = '127.0.0.1')  # Set as unknown (new) host
job.initConnection()

Connecting to machine: localhost at 127.0.0.1
Password for machine? ········
Testing connection...
bemo
Connected OK
Command exited with status 0.
=== stdout ===
bemo

(no stderr)


In [3]:
result = job.c.run('/home/femtolab/python/epsman/shell/conda_test.sh')
result

Testing Conda...
/usr/bin/python
/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/games:/usr/local/games


/home/femtolab/python/epsman/shell/conda_test.sh: line 15: conda: command not found
/home/femtolab/python/epsman/shell/conda_test.sh: line 27: conda: command not found


UnexpectedExit: Encountered a bad command exit code!

Command: '/home/femtolab/python/epsman/shell/conda_test.sh'

Exit code: 127

Stdout: already printed

Stderr: already printed



In [2]:
job = em.epsJob(IP = '10.8.0.10')  # Set as unknown (new) host
job.initConnection()

Connecting to machine: None at 10.8.0.10
User name for machine? paul
Password for machine? ········
Testing connection...
jake
Connected OK
Command exited with status 0.
=== stdout ===
jake

(no stderr)


Setting host dir tree.
Found multiple ePS directories, please select working dir:
0: ePS
1: ePS_AntonJr
2: ePS_results
List item #: 0
Set remote wrkdir: /home/paul/ePS

***Default paths set
{'IP': '10.8.0.10',
 'condaEnv': 'base',
 'condaPath': PosixPath('/home/paul/anaconda3/bin/activate'),
 'ePSpath': PosixPath('/opt/ePolyScat.E3/bin/ePolyScat'),
 'home': PosixPath('/home/paul'),
 'host': 'jake',
 'jobComplete': PosixPath('/home/paul/ePS/jobs/completed'),
 'jobPath': PosixPath('/home/paul/ePS/jobs'),
 'scpdir': PosixPath('/home/paul/ePS/scripts2019'),
 'wrkdir': PosixPath('/home/paul/ePS')}

***Checking bin dirs
ePolyScat bin OK
Anaconda bin OK


In [8]:
result = job.c.run('echo $PATH')
result

/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/games:/usr/local/games


<Result cmd='echo $PATH' exited=0>

In [3]:
# result = job.c.run('conda env list')
result = job.c.run('which python')

/usr/bin/python


In [11]:
result = job.c.run('~/ePS/epsman/shell/conda_test.sh')
result

Testing Conda...


/home/paul/ePS/epsman/shell/conda_test.sh: line 15: conda: command not found


/usr/bin/python


/home/paul/ePS/epsman/shell/conda_test.sh: line 27: conda: command not found


/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/games:/usr/local/games


UnexpectedExit: Encountered a bad command exit code!

Command: '~/ePS/epsman/shell/conda_test.sh'

Exit code: 127

Stdout: already printed

Stderr: already printed



In [12]:
job.c

<Connection host=10.8.0.10 user=paul>

In [7]:
# Try running as fabric task
# https://stackoverflow.com/questions/54268390/how-to-deploy-my-conda-env-to-a-vps-using-fabric-or-othervise
# This seems to work, just requires setting the Anaconda path

from fabric import task

# @task(hosts=["servername"])  # Not sure what the @tasks adds here - seems to be optional, but adds some capabilities.
# http://docs.pyinvoke.org/en/latest/api/tasks.html#invoke.tasks.task
def do_things(c):
    c.run('which python')
#     c.run('conda env list')  # This will fail
    with c.cd('~/ePS/epsman/shell'):
        # assuming you already added myenv to your path 
#         with c.prefix('source ~/anaconda3/bin/activate ePSproc-v1.2'): 
        with c.prefix(f"source {job.hostDefn[job.host]['condaPath']} ePSproc-v1.2"): 
            c.run('conda env list')
            c.run('ls')
            c.run('./conda_test.sh')
            c.run('which python')
            
do_things(job.c)

/usr/bin/python
# conda environments:
#
base                     /home/paul/anaconda3
ePSproc-v1.2          *  /home/paul/anaconda3/envs/ePSproc-v1.2

conda_note.txt
conda_test.sh
ePS_batch_job.sh
ePS_batch_nohup.sh
ePS_input_write_template_basic.sh
ePS_input_write_template_wf_sph.sh
jr_epsProc_nb.sh
nbHeaderRun.sh
Testing Conda...
/home/paul/anaconda3/envs/ePSproc-v1.2/bin/python
# conda environments:
#
base                     /home/paul/anaconda3
ePSproc-v1.2          *  /home/paul/anaconda3/envs/ePSproc-v1.2

/home/paul/anaconda3/envs/ePSproc-v1.2/bin:/home/paul/anaconda3/condabin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/games:/usr/local/games
# conda environments:
#
base                     /home/paul/anaconda3
ePSproc-v1.2          *  /home/paul/anaconda3/envs/ePSproc-v1.2

/home/paul/anaconda3/envs/ePSproc-v1.2/bin/python


In [ ]:
dir(job.c)

In [23]:
job.c.user

'paul'

In [29]:
result = job.c.run('[ -f "nohup.out" ]', warn = True, hide = True)  # Test for destination file, will return True if exists
destTest = result.ok
destTest

True

In [41]:
with job.c.cd('~/ePS/'):
    print(job.c.cwd)
    
print(job.c.cwd)

~/ePS/



In [5]:
job.checkFiles(['nohup.out', 'nofile.txt'], verbose = True)

nohup.out: True
nofile.txt: False


[True, False]

In [4]:
job.hostDefn

{'localhost': {'host': 'bemo',
  'IP': '127.0.0.1',
  'home': PosixPath('/home/femtolab'),
  'wrkdir': PosixPath('/home/femtolab/python/epsman/tests')},
 'jake': {'host': 'jake',
  'IP': '10.8.0.10',
  'home': PosixPath('/home/paul'),
  'wrkdir': PosixPath('/home/paul/ePS'),
  'scpdir': PosixPath('/home/paul/ePS/scripts2019'),
  'jobPath': PosixPath('/home/paul/ePS/jobs'),
  'jobComplete': PosixPath('/home/paul/ePS/jobs/completed'),
  'ePSpath': PosixPath('/opt/ePolyScat.E3/bin/ePolyScat'),
  'condaPath': PosixPath('/home/paul/anaconda3/bin/activate')}}

In [4]:
print(*job.hostDefn[job.host], sep='\n')
print(job.hostDefn[job.host])

host
IP
home
wrkdir
scpdir
jobPath
jobComplete
ePSpath
condaPath
condaEnv
{'host': 'jake', 'IP': '10.8.0.10', 'home': PosixPath('/home/paul'), 'wrkdir': PosixPath('/home/paul/ePS'), 'scpdir': PosixPath('/home/paul/ePS/scripts2019'), 'jobPath': PosixPath('/home/paul/ePS/jobs'), 'jobComplete': PosixPath('/home/paul/ePS/jobs/completed'), 'ePSpath': PosixPath('/opt/ePolyScat.E3/bin/ePolyScat'), 'condaPath': PosixPath('/home/paul/anaconda3/bin/activate'), 'condaEnv': 'base'}


In [5]:
import pprint
pprint.pprint(job.hostDefn[job.host])

{'IP': '10.8.0.10',
 'condaEnv': 'base',
 'condaPath': PosixPath('/home/paul/anaconda3/bin/activate'),
 'ePSpath': PosixPath('/opt/ePolyScat.E3/bin/ePolyScat'),
 'home': PosixPath('/home/paul'),
 'host': 'jake',
 'jobComplete': PosixPath('/home/paul/ePS/jobs/completed'),
 'jobPath': PosixPath('/home/paul/ePS/jobs'),
 'scpdir': PosixPath('/home/paul/ePS/scripts2019'),
 'wrkdir': PosixPath('/home/paul/ePS')}


In [7]:
testDict = {}

In [12]:
# testDict['key1'] = {'info':1, 'more':2}
testDict.update({1:{'info':1, 'more':2}})

In [13]:
testDict

{'key1': {'info': 1, 'more': 2},
 '1': {'info': 1, 'more': 2},
 1: {'info': 1, 'more': 2}}

In [15]:
testDict['1']

{'info': 1, 'more': 2}

In [18]:
hasattr(testDict.keys(),'1')

False

In [21]:
'1' in testDict.keys()

True